In [2]:
:option no-lint
:option no-show-types
import Data.List
import Control.Monad
{-# LANGUAGE AllowAmbiguousTypes #-}

# Functors & Monads

## Functors

> Functors are **structure-preserving** maps between **categories**.

![Functors](images/functor.jpeg)

* So what is connected in one category will be connected (**mapped**) in the other category
* Also function __composition__ will be preserved
* And __identity__ morphisms will be mapped between categories

## In pictures

Suppose we have a value
![Functors](images/fs-value.png)

We now how to apply a function, for example (+3), to this value
![Functors](images/fs-value_apply.png)

Now Suppose that this values is inside a **box** (a _context_). Like `Just 2` or `[2]` or `Right 2` or `IO a`
![Functors](images/fs-value_and_context.png)

Now we cannot apply a normal function to it.
![Functors](images/fs-no_fmap_ouch.png)

## In Code

In [3]:
(+3) 2

5

In [4]:
(+3) $ Just 2

: 

## What we need is to apply the function **inside** the `Functor`

### In Pictures again.
![Functors](images/fs-fmap_just.png)

In [5]:
fmap (+3) (Just 2)

Just 5

### How is it defined

```haskell
class Functor f where  
    fmap :: (a -> b) -> f a -> f b  
    (<$>) a b = fmap a b
        
```    

#### Maybe

```haskell
instance Functor Maybe where  
    fmap f (Just x) = Just (f x)  
    fmap f Nothing = Nothing  
```   

In [6]:
(+4) <$> Just 10
(+4) <$> Nothing

Just 14

Nothing

### Either
```haskell
instance Functor Either where  
    fmap f (Right x) = Right (f x)  
    fmap f (Left  x) = Left  x
```   

In [7]:
-- Using `fmap` (map over a functor)
fmap (+3) $ Just 2

-- Or use <$> 
(+3) <$> Just 2

(+4) <$> Right 10
(+4) <$> Left "Error"

Just 5

Just 5

Right 14

Left "Error"

## Others

#### Maps

In [8]:
import Data.Map as Map
fruits = Map.fromList[("Apples", 100), ("Oranges", 10)]
fruits
(+200) <$> fruits

fromList [("Apples",100),("Oranges",10)]

fromList [("Apples",300),("Oranges",210)]

#### Defining our own

In [9]:
data Tree a = Nil | Node a (Tree a) (Tree a)  deriving (Eq, Show)

instance Functor Tree where  
   fmap f Nil          = Nil
   fmap f (Node x l r) = Node (f x) (fmap f l) (fmap f r)

In [10]:
t = Node 4 (Node 3 Nil Nil) (Node 5 Nil Nil)
t
(+2) <$> t
sqrt <$> t

Node 4 (Node 3 Nil Nil) (Node 5 Nil Nil)

Node 6 (Node 5 Nil Nil) (Node 7 Nil Nil)

Node 2.0 (Node 1.7320508075688772 Nil Nil) (Node 2.23606797749979 Nil Nil)

## Monads

* In Algebra, more specifically in _Category Theory_ **Monads** are an algebraic structure.
* A _monad_ is just a _monoid_ in the _category_ of _endofunctors_.

* Just don't care about that.
* In **functional programming**, a **monad** is a **design pattern**.
* They describe a way to combine **computations** between **'enriched'** values.

## **Enriched** values ?

How do you enriched a value ? You put it into a **"box"**.

Actually the **box** is a **context** that adds additional info to the value.

![](images/fs-value_and_context.png)

So, yes a `Monad` is a `Functor`, but know we want to do something more

What Types of **contexts**:
* `Maybe` : To represent a potentially missing value.
* `Either`: To represent a successful operation or a failure.
* `IO`: To represent side-effects
* `State` : To attach state to a chain of computations.
* `Future`: An asynchronous computation
* `Reader`: To store shared configuration values
* `Writer`: To accumulate values, for example to do logging.
* List and other collections
* Big etc....

## The problem

What if the function we eant to apply **returns** a Monad.

In [11]:
half::Int -> Maybe Int
half x                                 -- Divide a number by 2. Succeeds ONLY if it is even
    | even x    = Just (x `div` 2)
    | otherwise = Nothing
    
half 3
half 6

Nothing

Just 3

![Half](images/half.png)

## The problem

In [12]:
half $ half 24


: 

![HalfOuch](images/half_ouch.png)

In [13]:
half <$> half 24

Just (Just 6)

In [14]:
half 24 >>= half -- '>>=' is called 'flatMap' in other languages

Just 6

## Monad Definition

A Monad definition consists of **3** things
```haskell
class Monad m where                 --<-- The Monad Type definition
 return :: a -> m a                 --<-- A function that takes a value and
                                          wraps it into the Monad
 
 (>>=) :: m a -> (a -> m b) -> m b  --<-- A function that chains 2 executions
```    

![Bind](images/bind_def.png)

In [15]:
half 6 
half 6 >>= half
half 24 >>= half >>= half

Just 3

Nothing

Just 3

## Maybe as a Monad

The definition of `Maybe` as a monad:
```haskell
instance Monad Maybe where
    return x         = Just x
    Nothing  >>= fun = Nothing
    Just val >>= fun = fun val
```    

## More examples

In [48]:
l = [1, 2, 3]

tail $ tail $ tail $ tail l  

: 

In [17]:
safeTail :: [a] -> Either String [a]
safeTail []     = Left "Trying to get tail from an empty list"
safeTail (_:xs) = Right xs


safeTail l  >>= safeTail >>= safeTail >>= safeTail >>= safeTai

Left "Trying to get tail from an empty list"

In [18]:
safeHalf :: Int -> Either String Int
safeHalf n                                 -- Divide a number by 2. Succeeds ONLY if it is even
    | even n    = Right (n `div` 2)
    | otherwise = Left  ("Error: '" ++ show n ++ "' is an odd number.")
    
safeHalf 20 >>= safeHalf >>= safeHalf >>= safeHalf  

Left "Error: '5' is an odd number."

## The `do` notation

Suppose we want to create a `halfTwice` function

In [19]:
halfTwice n = safeHalf n >>= safeHalf
halfTwice 6

Left "Error: '3' is an odd number."

In [20]:
halfTwice n = safeHalf n >>= (\m -> safeHalf m)
halfTwice 6

Left "Error: '3' is an odd number."

In [21]:
halfTwice n = do
                m <- safeHalf n
                safeHalf m                
halfTwice 6                

Left "Error: '3' is an odd number."

In [22]:
f n = do m <- safeHalf n
         safeHalf (m+1)   

## The `do` notation

`sumHalfs` function

In [23]:
sumHalfs:: Int -> Int -> Either String Int 
sumHalfs n m = do
        x <- safeHalf n  
        y <- safeHalf m
        return (x + y)
sumHalfs 20 10

Right 15

In [24]:
sumHalfs n m = safeHalf n  >>= (\x ->
               safeHalf m  >>= (\y ->
               return (x + y))) 
sumHalfs 20 10               

Right 15

## IO is a Monad

```haskell
do 
    x <- getLine
    putStrln (reverse x) 
```

Gets translated to

```haskell
getLine >>= (\x -> putStrLn (reverse x))
```

## Ignoring the result of the first expression

There is another operator in the Monad definition, that chains the executions ignoring the value of the first expression (it just returns the second expression).
```haskell
(>>) :: m a -> m b -> m b
m1 >> m2 = m1 >>= (\_ -> m2) 
```

```haskell
do 
    putStrLn "Hello"
    putStrLn "World!"
```

Is translated to:

```haskell
putStrLn "Hello" >> putStrLn "World!"
```

## What is different about IO ?

> There is no way to get a value **out** of the IO monad


For example with `Maybe` you can do, something like:

```haskell
add:: Maybe Int -> Maybe Int -> Int
add (Just x) (Just y) = x + y
```

But with IO....

In [25]:
add::IO Int -> IO Int -> Int
add (IO x) (IO y) = x + y 

: 

## What you can do is:


In [26]:
add::IO Int -> IO Int -> IO Int
add a b = do 
    x <- a
    y <- b
    return (x+y)

So... **_What Happens in IO, Stays in IO_**

## Other Monad Operations 
### sequence 
`sequence :: (Monad m) => t (m a) -> m (t a)`

Evaluate each monadic action in the structure from left to right, and collect the results


In [27]:
sequence [Just 1, Just 2, Just 4]
sequence [Just 1, Nothing, Just 2]
sequence [Right 1, Left "xxx", Right 4]

Just [1,2,4]

Nothing

Left "xxx"

In [28]:
--   sequence [Just 1, Just 2, Just 4] it is the same as:
do 
    x <- Just 1
    y <- Just 2
    z <- Just 4
    return [x, y, z] 

Just [1,2,4]

**`sequence_`** same as `sequence` but results are not collected

In [ ]:

sequence_ [print 1, putStrLn "Hello", print 3]


1
Hello
3

## Other Monad Operations

`mapM :: (Monad m) => (a -> m b) -> t a -> m (t b)`

Map each element of a structure (for example a list) to a monadic action.

And then evaluate these actions from left to right, and collect the results.

It is equivalent to `sequence . map`

### Example. Printing a list of words:

In [30]:
printList [] = return ()
printList (x:xs) =
    do 
       putStrLn x
       printList xs       

In [31]:
mapM putStrLn ["The","quick","brown","fox"]

The
quick
brown
fox
[(),(),(),()]

In [32]:
mapM_ putStrLn ["Hello", "World"]

Hello
World

## Other Monad Operations

```haskell
forM_ :: (Monad m) =>  [a] -> (a -> m b) -> m ()
forM_ l f = mapM_ f l
```
(Just Flip `mapM_` arguments)


In [33]:
fruits = ["Apples", "Bananas", "Oranges"]

forM_ fruits $ \fruit -> do
    putStrLn fruit
    putStrLn "-----"

Apples
-----
Bananas
-----
Oranges
-----

`filterM :: (Monad m) => (a -> m Bool) -> [a] -> m [a]`

`foldM :: (Monad m) => (a -> b -> m a) -> a -> [b] -> m a`

## The Writer Monad


`Writer` is a `Monad` that enriches a value with additional information that you can write while doing your computations.

Like attaching a notebook to the box.

Its type is something like:

`type Writer w v = Writer (v, w)`  

Where `w` is the object you write things to and `v` is the your value.

You can create a writer with function:

`writer:: (v, w) -> Writer w v` For example:

`writer (10, "This is the price.")`

## The Writer Monad

#### Example

In [47]:
import Control.Monad.Writer

addWithLog:: Int -> Int -> Writer String Int
addWithLog x y = writer (x+y, "Just adding: " ++ show x ++ " to " ++ show y)

s = addWithLog 10 20
runWriter s

(30,"Just adding: 10 to 20")

Then to extract the value from the writer you invoke the function:

`runWriter :: Writer w a -> (a, w)`

In [35]:
runWriter s

fst $ runWriter s
snd $ runWriter s 

(30,"Just adding: 10 to 20")

30

"Just adding: 10 to 20"

## Using for logging

#### Example

In [36]:
hcf :: Int -> Int -> Int  
hcf a b   
    | b == 0    = a  
    | otherwise = hcf b (a `mod` b)
hcf 24 27    

3

Want to add some logging, so I make it return a `Writer`

In [37]:
hcf :: Int -> Int -> Writer [String] Int  
hcf a b   
    | b == 0    = do
                      return a  
    | otherwise = hcf b (a `mod` b)      

runWriter (hcf 24 27) 

(3,[])

In [38]:
hcf :: Int -> Int -> Writer [String] Int  
hcf a b   
    | b == 0    = do
                     writer (a, ["Return: " ++ show a])  
    | otherwise = hcf b (a `mod` b)      

runWriter (hcf 24 27)  

(3,["Return: 3"])

## Using Tell

I can use the function: `tell` that receives the value I want to `write` in the context of a `Writer`

The Info will be added to the _notebook_.



In [39]:
hcf :: Int -> Int -> Writer [String] Int  
hcf a b   
    | b == 0    = do
      tell ["Return: " ++ show a]
      return a  
    | otherwise = do
      tell [show a ++ " mod " ++ show b ++ " = " ++ show (a `mod` b)] 
      hcf b (a `mod` b)       

In [40]:
runWriter $ hcf 24 27

(3,["24 mod 27 = 24","27 mod 24 = 3","24 mod 3 = 0","Return: 3"])

In [41]:
fst $ runWriter $ hcf 24 27

3

In [42]:
mapM_ putStrLn $ snd $ runWriter $ hcf 24 27

24 mod 27 = 24
27 mod 24 = 3
24 mod 3 = 0
Return: 3

## Add some Info


In [43]:
hcf :: Int -> Int -> Writer [String] Int 
hcf a b  = do
    tell [""] 
    tell ["hcd " ++ show a ++ " " ++ show b] 
    if b == 0 then  do
      tell ["Return: " ++ show a]
      return a  
    else do
      tell [show a ++ " mod " ++ show b ++ " = " ++ show (a `mod` b)] 
      hcf b (a `mod` b)
 

In [44]:
mapM_ putStrLn $ snd $ runWriter $ hcf 24 27


hcd 24 27
24 mod 27 = 24

hcd 27 24
27 mod 24 = 3

hcd 24 3
24 mod 3 = 0

hcd 3 0
Return: 3